# Initial Data Preparation

## Objectives

* Fetch data from Kaggle and save as raw data in organised folders
* Initial data preparation and data cleaning - removing non image files
* Read COCO annotations of second dataset to split into good and bad quality lemons
* Combine two datasets into a single one for analysis
* Perform image focal point isolation and background cropping tasks to uniformise dataset
* Split data into Train, Validation, Test sets

## Inputs

* Kaggle JSON file - authentication token

## Outputs

* Generate Lemon Quality Dataset, split into Train, Validation, and Test sets



---

# Change working directory

Change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/lemon-qualitycontrol/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/lemon-qualitycontrol'

## Obtain and save data from Kaggle API

Install Kaggle

In [4]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 28.9 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=9885b7b79b2b9a77dfa39c996f19e018dbdcff38207fcb4fb59e2aa748aed765
  Stored in directory: /home/gitpod/.cache/pip/wheels/03/f3/c7/fc5a63bb33d22177609b06c5b4c714b5eb3f1b195ce9dc5e47
Successfully built kaggle

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


Set Kaggle config directory environment variable to that of current working directory and set authentication to 600 to allow Kaggle package to locate JSON file

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set KaggleDataset variable as the [URL](https://www.kaggle.com/datasets/yusufemir/lemon-quality-dataset) for the dataset on Kaggle and create destination folder variable for it to be downloaded into.
Run Kaggle command to download dataset into destination folder. Repeat process for the second Kaggle dataset at the following [URL](https://www.kaggle.com/datasets/maciejadamiak/lemons-quality-control-dataset)

In [19]:
KaggleDataset = "yusufemir/lemon-quality-dataset"
DestinationFolder = "inputs/lemon-quality-dataset"
! kaggle datasets download -d {KaggleDataset} -p {DestinationFolder}

KaggleDataset2 = "maciejadamiak/lemons-quality-control-dataset"
DestinationFolder2 = "inputs/lemon-quality-dataset-2"
! kaggle datasets download -d {KaggleDataset2} -p {DestinationFolder2}


100%|███████████████████████████████████████▉| 232M/233M [00:06<00:00, 47.2MB/s]
100%|████████████████████████████████████████| 233M/233M [00:06<00:00, 39.7MB/s]
 98%|█████████████████████████████████████ | 81.0M/83.0M [00:03<00:00, 31.1MB/s]
100%|██████████████████████████████████████| 83.0M/83.0M [00:03<00:00, 23.7MB/s]


Unzip downloaded files and subsequently delete the originally downloaded zipped files

In [20]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/lemon-quality-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/lemon-quality-dataset.zip')

with zipfile.ZipFile(DestinationFolder2 + '/lemons-quality-control-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder2)

os.remove(DestinationFolder2 + '/lemons-quality-control-dataset.zip')

---

# Data Preparation

Set input dataset paths for first and second lemon dataset

In [9]:
DataPath = "inputs/lemon-quality-dataset/lemon_dataset"
DataPath2 = 'inputs/lemon-quality-dataset-2/data/lemon-dataset/lemon-dataset'

Import COCO API and use it to read dataset

In [10]:
from pycocotools.coco import COCO
coco = COCO(f'{DataPath2}/annotations/instances_default.json')
cats = coco.cats
info = coco.info()
print(cats)
print(info)



loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
year: 2020
url: https://github.com/softwaremill/lemon-dataset
version: 1
date_created: 28.07.2020
contributor: Maciej Adamiak
institution: SoftwareMill
description: Lemons quality control dataset
{1: {'id': 1, 'supercategory': '', 'name': 'image_quality'}, 2: {'id': 2, 'supercategory': '', 'name': 'illness'}, 3: {'id': 3, 'supercategory': '', 'name': 'gangrene'}, 4: {'id': 4, 'supercategory': '', 'name': 'mould'}, 5: {'id': 5, 'supercategory': '', 'name': 'blemish'}, 6: {'id': 6, 'supercategory': '', 'name': 'dark_style_remains'}, 7: {'id': 7, 'supercategory': '', 'name': 'artifact'}, 8: {'id': 8, 'supercategory': '', 'name': 'condition'}, 9: {'id': 9, 'supercategory': '', 'name': 'pedicel'}}
None


Remove all lemons with a pedicel in the foreground

In [11]:
##Pseudo code: delete all lemons that have a prominent pedicel
##

In [12]:
## Run function to delete all lemons with pedicel

1245
1445
Lemons successfully sorted


Separate lemons into lists of healthy and unhealthy lemons, decided by passing tags for filtering dataset in. Here, illness, gangrene, and mould are chosen as markers for unhealthy lemons, and the lemons on which said traits were annotated are separated from those on which they were not.

In [8]:
def sort_lemons(coco, tags: list) -> tuple:
    all_lemon_ids = coco.getImgIds()
    temp_list = []
    bad_ids = []
    good_ids = []
    for tag in tags:
        arr = coco.getImgIds(catIds=[tag])
        for i in arr:
            temp_list.append(i)
    
    [bad_ids.append(i) for i in temp_list if i not in bad_ids]
    [good_ids.append(i) for i in all_lemon_ids if i not in bad_ids]
    bad_lemons = coco.loadImgs(ids=bad_ids)
    good_lemons = coco.loadImgs(ids=good_ids)
    print(len(bad_lemons))
    print(len(good_lemons))
    return bad_lemons, good_lemons


In [9]:
my_sort = sort_lemons(coco=coco, tags=[2,3,4,5,6])
print('Lemons successfully sorted')

2566
124
Lemons successfully sorted


Move sorted image files into folders with first lemon dataset

In [8]:
import shutil
def move_sorted_images(sort: tuple):
    for i in sort[0]: #improve this section of code to iterate through both labels at once
        file_ext = i['file_name']
        image_file = f'{DataPath2}/{file_ext}'                             
        shutil.move(image_file, f'{DataPath}/bad_quality')
    for i in sort[1]:
        file_ext = i['file_name']
        image_file = f'{DataPath2}/{file_ext}'
        shutil.move(image_file, f'{DataPath}/good_quality')

    

In [11]:
my_move = move_sorted_images(sort=my_sort)


## Data Cleaning

Remove second lemon quality dataset folder, along with empty background folder and git hooks folder in first dataset

In [12]:
shutil.rmtree('inputs/lemon-quality-dataset-2/')
shutil.rmtree(f'{DataPath}/empty_background/')
shutil.rmtree(f'{DataPath}/.git/')

Check for and remove non-image files

In [13]:
def remove_non_image_file(my_data_dir: str):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) 
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))
    
    

In [14]:
remove_non_image_file(my_data_dir=DataPath)

Folder: bad_quality - has image file 3517
Folder: bad_quality - has non-image file 0
Folder: good_quality - has image file 1249
Folder: good_quality - has non-image file 0


---

## Remove Image Backgrounds

Install rembg package

In [15]:
!pip install rembg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 102.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 44.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow-cpu 2.6.0 requires typing-extensions~=3.7.4, but

Create directories for processed images

In [16]:
BadQualityPath = f"{DataPath}/bad_quality"
GoodQualityPath = f"{DataPath}/good_quality"
os.makedirs(f"{DataPath}/bad_quality_processed")
os.makedirs(f"{DataPath}/good_quality_processed")
BadQualityProcessed = f"{DataPath}/bad_quality_processed"
GoodQualityProcessed = f"{DataPath}/good_quality_processed"

Remove image background to isolate lemon

In [17]:
from rembg import remove
from PIL import Image
import matplotlib.pyplot as plt



good_quality_files = os.listdir(GoodQualityPath)
for index, my_file in enumerate(good_quality_files):
    image=Image.open(f'{GoodQualityPath}/{my_file}')
    removed = remove(image)
    imageBox = removed.getbbox()
    cropped = removed.crop(imageBox)
    cropped.save(f'{GoodQualityProcessed}/{index}.png')

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

SystemError: <built-in function __import__> returned a result with an error set

Downloading...
From: https://drive.google.com/uc?id=1tCU5MM1LhRgGou5OpmpjBQbSrYIUoYab
To: /home/gitpod/.u2net/u2net.onnx
100%|██████████| 176M/176M [00:00<00:00, 201MB/s]  


In [ ]:
bad_quality_files = os.listdir(BadQualityPath)
for index, my_file in enumerate(bad_quality_files):
    image=Image.open(f'{BadQualityPath}/{my_file}')
    removed = remove(image)
    imageBox = removed.getbbox()
    cropped = removed.crop(imageBox)
    cropped.save(f'{BadQualityProcessed}/{index}.png')

Reinstall requirements.txt to avoid dependency conflicts

In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 116.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
starlette 0.19.1 requires typing-extensions>=3.10.0; python_version < "3.10", but you have typing-extensions 3.7.4.3 which is incompatible.
rembg 2.0.25 requi

Remove files of unprocessed images and rename processed image files

In [9]:
shutil.rmtree(f'{DataPath}/bad_quality/')
shutil.rmtree(f'{DataPath}/good_quality/')

In [ ]:
os.rename(f'{DataPath}/bad_quality_processed/', f'{DataPath}/bad_quality/')
os.rename(f'{DataPath}/good_quality_processed/', f'{DataPath}/good_quality/')

## Perform train-validation-test split on data

Define train-test-validation split function

In [ ]:
import random

def split_train_validation_test_images(my_data_dir: str, train_set_ratio: float, validation_set_ratio: float, test_set_ratio: float):
  
  if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
    print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
    return

  labels = os.listdir(my_data_dir) 
  if 'test' in labels:
    pass
  else: 
    for folder in ['train','validation','test']:
      for label in labels:
        os.makedirs(name=my_data_dir+ '/' + folder + '/' + label)

    for label in labels:

      files = os.listdir(my_data_dir + '/' + label)
      random.shuffle(files)

      train_set_files_qty = int(len(files) * train_set_ratio)
      validation_set_files_qty = int(len(files) * validation_set_ratio)

      count = 1
      for file_name in files:
        if count <= train_set_files_qty:
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/train/' + label + '/' + file_name)
          

        elif count <= (train_set_files_qty + validation_set_files_qty ):
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/validation/' + label + '/' + file_name)

        else:
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                  my_data_dir + '/test/' +label + '/'+ file_name)
          
        count += 1

      os.rmdir(my_data_dir + '/' + label)
    

Apply function to combined lemon dataset

In [ ]:
split_train_validation_test_images(my_data_dir = DataPath,
                            train_set_ratio = 0.7,
                            validation_set_ratio = 0.1,
                            test_set_ratio = 0.2)